# PageRank

In this assignment, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs (NOT the power method with explicit matrix multiplication).

Consider the following directed graph:

![A directed link graph](https://ccs.neu.edu/home/dasmith/courses/cs6200/pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [1]:
small_nodes = [(0, 'A'),
        (1, 'B'),
        (2, 'C'),
        (3, 'D'),
        (4, 'E'),
        (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [2]:
small_edges = [
 (0, 1),
 (0, 2),
 (0, 5),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 2),
 (3, 4),
 (3, 5),
 (4, 0),
 (5, 0),
 (5, 1),
 (5, 4)]

We use integer identifiers for the nodes for efficiency. In most real-world collections of hyperlinks, unlike this example, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm we discussed in class. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [3]:
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.
import numpy as np

def page_rank_fixed_iter(nodes, edges, iterations=10):
  outboundLinks = {}
  for singleNode in nodes:
    outboundLinks[singleNode[0]] = []
  for singleEdge in edges:
    if singleEdge[0] in outboundLinks.keys():
      outboundLinks[singleEdge[0]].append(singleEdge[1])

  P = len(nodes)
  I = np.ones(P) / P
  
  for i in range(iterations):
    R = np.ones(P) * (0.15 / P)
    
    total = 0
    for singleNode in nodes:
      Q = outboundLinks[singleNode[0]]
      calc = 0
      if len(Q) > 0:
        for linkNode in Q:
          R[linkNode] = R[linkNode] + (1 - 0.15) * I[singleNode[0]] / len(Q)
      else:
        calc = calc + 1
      total = total + calc * ((1 - 0.15) * I[singleNode[0]] / P)
    
    for singleNode in nodes:
      R[singleNode[0]] = R[singleNode[0]] + total
    I = R[:]
  
  result = R[:]
  R = {}
  for ele in range(len(result)):
    R[nodes[ele][1]] = result[ele]
  return R



In [4]:
# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
print(page_rank_fixed_iter(small_nodes, small_edges, 1))
print(page_rank_fixed_iter(small_nodes, small_edges, 10))
print(page_rank_fixed_iter(small_nodes, small_edges, 100))

{'A': 0.24930555555555556, 'B': 0.11944444444444444, 'C': 0.14305555555555555, 'D': 0.13124999999999998, 'E': 0.21388888888888885, 'F': 0.14305555555555555}
{'A': 0.25203637602817186, 'B': 0.13930650918251075, 'C': 0.15129376593475066, 'D': 0.11896297277689877, 'E': 0.18710661014291707, 'F': 0.15129376593475066}
{'A': 0.25212710537519556, 'B': 0.13930618531853856, 'C': 0.15130648986670525, 'D': 0.11890782257353917, 'E': 0.18704590699931606, 'F': 0.15130648986670525}


## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [5]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

--2022-10-18 18:29:50--  https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703486 (3.5M) [application/x-gzip]
Saving to: ‘vertices-edu.txt.gz’

vertices-edu.txt.gz 100%[===================>]   3.53M  --.-KB/s    in 0.1s    

2022-10-18 18:29:51 (25.6 MB/s) - ‘vertices-edu.txt.gz’ saved [3703486/3703486]

--2022-10-18 18:29:51--  https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12829526 (12M) [application/x-gzip]
Saving to: ‘edges-edu.txt.gz’

edges-edu.txt.gz    100%[===================>]  12.23M  59.4MB/s    in 0.2s    

2022-10-18 18:29:51 (59.4 MB/s) - ‘edges-edu.txt.gz’ saved [12829526/128295

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [6]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.
import string
big_nodes = []
big_edges = []

f1 = open('vertices-edu.txt')
f2 = open('edges-edu.txt')

for line in f1:
  tmp = line.split(' ')
  big_nodes.append((int(tmp[0]),tmp[1].strip('\n')))

for line in f2:
  tmp = line.split(' ')
  big_edges.append((int(tmp[0]),int(tmp[1].strip('\n'))))

print(big_nodes[:10])
print(big_edges[:10])

[(0, 'edu.00zl5e'), (1, 'edu.06hxbt'), (2, 'edu.082ifc'), (3, 'edu.083mjs'), (4, 'edu.09xzrr'), (5, 'edu.0aoqqj'), (6, 'edu.0ax4el'), (7, 'edu.0c5fez'), (8, 'edu.0cosn2'), (9, 'edu.0dcdp8')]
[(386, 440), (19202, 1033), (103884, 2635), (342306, 7399), (8366, 8312), (8358, 8312), (8949, 8987), (8982, 8987), (8910, 8987), (9028, 8999)]


Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall the definition of entropy from our discussion of data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [11]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.
import numpy as np
import math
from itertools import count

def page_rank(nodes, edges, threshold=1):
  outboundLinks = {}
  for singleNode in nodes:
    outboundLinks[singleNode[0]] = []
  for singleEdge in edges:
    if singleEdge[0] in outboundLinks.keys():
      outboundLinks[singleEdge[0]].append(singleEdge[1])

  P = len(nodes)
  I = np.ones(P) / P
  begin = 0
  for i in count():
    R = np.ones(P) * (0.15 / P)
    
    total = 0
    for singleNode in nodes:
      Q = outboundLinks[singleNode[0]]
      calc = 0
      if len(Q) > 0:
        for linkNode in Q:
          R[linkNode] = R[linkNode] + (1 - 0.15) * I[singleNode[0]] / len(Q)
      else:
        calc = calc + 1 
      total = total + calc * ((1 - 0.15) * I[singleNode[0]] / P)

    for singelNode in nodes:
      R[singelNode[0]] = R[singelNode[0]] + total
    I = R[:]
    E = 0
    for singeNode in nodes:
      E = E + I[singeNode[0]] * math.log(I[singeNode[0]])
    perp = 2**(-E)
    print("%d iteration:" % (i + 1))
    print(perp)
    if abs(perp - begin) < threshold:
      break
    else:
      begin = perp
  result = R[:]
  R = {}
  for ele in range(len(result)):
    R[nodes[ele][1]] = result[ele]
  return R
      
# Run until perplexity changes by less than 1
PR = page_rank(big_nodes, big_edges, 1)

1 iteration:
6230.31110347081
2 iteration:
5602.781784251892
3 iteration:
5272.0703261129565
4 iteration:
5142.249454576224
5 iteration:
5072.052362184011
6 iteration:
5040.189409819244
7 iteration:
5020.550223015783
8 iteration:
5010.519028781535
9 iteration:
5003.593503260447
10 iteration:
4999.783093084004
11 iteration:
4996.84173936159
12 iteration:
4995.137058685129
13 iteration:
4993.70762958829
14 iteration:
4992.836003916483


## Link Analysis

\[20 points\] In this final section, you will compute some properties of the web-site graph you downloaded.

First, consider the _in-link count_ of a website, simply the number of web-sites pointing to it (including self-links). 

In [12]:
# TODO: List the document ID, domain name, and in-link count of the 60 websites with the highest in-link count
inboundLinks = {}
reverse_edges = []
for edge in big_edges:
  reverse_edges.append((edge[1],edge[0]))

inboundLinks = {}
for singleNode in big_nodes:
  inboundLinks[singleNode[0]] = []
for singleEdge in reverse_edges:
  if singleEdge[0] in inboundLinks.keys():
    inboundLinks[singleEdge[0]].append(singleEdge[1])

count = {}
for key,value in inboundLinks.items():
  count[key] = len(value)

count = sorted(count.items(), key=lambda x: x[1], reverse = True)
result = []
for countItem in count[:60]:
  result.append((big_nodes[countItem[0]],countItem[1]))
print(result)

[((185524, 'edu.mit.web'), 4388), ((278032, 'edu.stanford'), 4021), ((244433, 'edu.purdue.english.owl'), 3531), ((140443, 'edu.indiana'), 3339), ((237176, 'edu.princeton'), 3251), ((64587, 'edu.columbia'), 3123), ((465503, 'edu.yale'), 2804), ((418623, 'edu.utexas'), 2622), ((383763, 'edu.unc'), 2592), ((197698, 'edu.nap'), 2494), ((439637, 'edu.washington'), 2291), ((373442, 'edu.umich'), 2281), ((440674, 'edu.washington.depts'), 2276), ((148945, 'edu.jhu.muse'), 2255), ((60975, 'edu.colorado'), 2232), ((449738, 'edu.wisc'), 2230), ((38320, 'edu.bu'), 2205), ((83572, 'edu.dartmouth'), 1965), ((408380, 'edu.usc'), 1952), ((178879, 'edu.mit'), 1946), ((27307, 'edu.berkeley'), 1908), ((233405, 'edu.pitt'), 1857), ((191069, 'edu.msu'), 1810), ((326371, 'edu.uchicago.press'), 1763), ((136464, 'edu.illinois'), 1753), ((93874, 'edu.educause'), 1741), ((56979, 'edu.cmu.cs'), 1730), ((199032, 'edu.ncsu'), 1709), ((36294, 'edu.brown'), 1702), ((202182, 'edu.nd'), 1689), ((68675, 'edu.cornell'),

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [13]:
# TODO: List the document ID, domain name, and PageRank of the 60 websites with the highest PageRank.
resultPR = sorted(PR.items(), key=lambda x: x[1], reverse = True)
resultPR = resultPR[:60]
reverse_nodes = {}
for singleNode in big_nodes:
  reverse_nodes[singleNode[1]] = singleNode[0]

resultset = []
for singleItem in resultPR:
  resultset.append((reverse_nodes[singleItem[0]],singleItem[0],singleItem[1]))
print(resultset)

[(185524, 'edu.mit.web', 0.000928978898154372), (465503, 'edu.yale', 0.0006987101785944766), (278032, 'edu.stanford', 0.0005950595762236685), (318278, 'edu.ucdavis.cas', 0.0005802156798546758), (136464, 'edu.illinois', 0.0005799394526666204), (237176, 'edu.princeton', 0.0005024009789224997), (319795, 'edu.ucdavis.ice.code', 0.0004948607254235917), (284517, 'edu.stanford.web', 0.0004752332345921687), (383763, 'edu.unc', 0.00047227506899357135), (64587, 'edu.columbia', 0.00044779743971780925), (449738, 'edu.wisc', 0.00043631547091175084), (60975, 'edu.colorado', 0.000432757645949801), (346969, 'edu.ucsf', 0.0004299414187967938), (140443, 'edu.indiana', 0.00042987889225767033), (334739, 'edu.uconn', 0.0004127743141853516), (42502, 'edu.byu.cas', 0.00041210612656012645), (14945, 'edu.arizona', 0.0004092741252289774), (244433, 'edu.purdue.english.owl', 0.0004034198774626775), (373442, 'edu.umich', 0.0004021473017497311), (439637, 'edu.washington', 0.000398833345064602), (408380, 'edu.usc', 

Finally, compute some summary statistics on this dataset.

In [24]:
outboundLinks = {}
for singleNode in big_nodes:
  outboundLinks[singleNode[0]] = []
for singleEdge in big_edges:
  if singleEdge[0] in outboundLinks.keys():
    outboundLinks[singleEdge[0]].append(singleEdge[1])

In [25]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);
count = {}
for key,value in inboundLinks.items():
  count[key] = len(value)
calc = 0
for val in count.values():
  if val == 0:
    calc = calc + 1
print(calc / len(big_nodes))
# - the proportion of websites with no out-links (i.e., sink nodes);
calc = 0
for val in outboundLinks.values():
  if len(val) == 0:
    calc = calc + 1
print(calc / len(big_nodes))
# - the proportion of websites whose PageRank is higher than the initial uniform distribution.
calc = 0
begin = 1 / len(big_nodes) 
for val in PR.values():
  if val > begin:
    calc = calc + 1
print(calc / len(big_nodes))

0.26153633041013563
0.6106641661427643
0.1552953211077605
